# ALMA Project Trial Task: Collaborative Human–LLM Architecture

**Candidate:** El Heyba EL HEYBA  
**Context:** PhD Position in *Computer-Supported Collaborative Learning with Large Language Models (LLMs)*

---

## 1. Project Overview & Motivation

> *"Humans tend not to engage with LLMs collaboratively... they accept or reject initial responses rather than engage in collaborative work"*  
> — Gomez et al., 2025

This notebook addresses the core challenge identified in the ALMA project.
The goal of this application is to shift the Human–LLM dynamic from a **Master–Slave** relationship to a **Partner–Partner (Collaborative)** relationship.

This prototype implements a **"Collaborative Friction" protocol**.  
Instead of maximizing convenience (i.e., giving the answer immediately), the system maximizes **cognitive engagement** by forcing the user to:

- Verbalize their intent  
- Defend their design choices  
- Negotiate meaning  

before receiving assistance.

---

## 2. System Architecture

The application is built around a central class, `CollaborativePartner`, which acts as a **Cognitive Gatekeeper**.

### The Core Logic Flow

The architecture does not simply pass user input to the LLM.  
Instead, it **intercepts the interaction** to enforce a specific pedagogical protocol:

1. **Input Analysis** – The system evaluates the quality of the user's input:
   - *Lazy input* (e.g., `"Write code for X"`)  
     → **Trigger Refusal Mechanism**
   - *Ambiguous input* (e.g., `"Make it efficient"`)  
     → **Trigger Negotiation Mechanism**
   - *Thoughtful input* (e.g., `"I plan to use X because Y"`)  
     → **Trigger Optimization / Critique**

2. **The "Persona" Engine (System Prompt)**  
   The LLM is governed by a strict system instruction that overrides its default *helpful assistant* behavior.

   - **Rule of Neutrality**  
     Emotional reinforcement (e.g., praise) is stripped out to maintain focus on abstract logic.

   - **Rule of Inquiry**  
     The model is forbidden from answering *zero-shot* requests and must ask a clarifying question first.

   - **Adaptive Expertise**  
     A variable `collaboration_level` dynamically adjusts the model’s behavior:
     - **Novice Mode:** Scaffolding and guidance  
     - **Professional Mode:** *Devil’s Advocate* critique and optimization

### Technical Components

- **LLM Backend:** Google Gemini API  
  *(specifically `gemini-flash-latest` for speed and instruction-following reliability)*

- **Prompt Engineering:**  
  - Chain-of-Thought constraints  
  - Role-playing definitions

- **Python Logic:**  
  - Modular class structure  
  - Conversation history and state management

---

## 3. Key Features Demonstrated

This notebook is divided into interactive cells that demonstrate specific collaborative behaviors:

- **Reflective Questioning**  
  Forcing the user to clarify vague requests

- **Negotiating Meaning**  
  Aligning technical definitions (e.g., *“efficiency”*) before coding

- **Anti–Free-Rider Mechanism**  
  Refusing to generate code until the user provides a logical plan

- **Shared Problem Solving**  
  Optimizing user-generated ideas rather than replacing them


**Setup & API Connection**

Run this cell to install genretiveai library.

In [1]:
%pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


## Environment Setup & Robust API Handler

This cell initializes the connection to the Google Gemini API and defines a robust handler function `query_gemini` for managing the conversation flow.

**Key Technical Implementations:**
* **Model Selection:** Utilizes `gemini-flash-latest`, chosen for its balance of speed and instruction-following capability suitable for the Free Tier constraints.
* **Quota Management:** Implements an **Auto-Retry Mechanism** (exponential backoff strategy). If the application hits the Free Tier rate limit (HTTP 429 "Quota Exceeded"), the system automatically pauses for 60 seconds and retries rather than crashing.
* **History Adaptation:** Converts our custom "User/System" role format into the specific dictionary structure required by the Gemini API (`role='user'` vs `role='model'`).
* **System Instructions:** Natively passes the "Persona" rules (Collaborative Friction logic) to the model instance for every interaction.

### ⚠️ IMPORTANT: API Key Configuration Required

**To run this notebook, you must provide your own Google Gemini API Key.**

This application uses Google's Generative AI (Gemini) models. While the Free Tier is generous, it requires authentication.

**How to Get Your Free API Key:**
1.  Go to **[Google AI Studio](https://aistudio.google.com/)**.
2.  Sign in with your Google Account.
3.  Click the **"Get API Key"** button on the top left.
4.  Click **"Create API Key"** (you can create it in a new project).
5.  **Copy** the key string.

**How to Use It:**
1.  Go to the **Setup & Authentication** cell below.
2.  Find the line: `my_api_key = "PASTE_YOUR_KEY_HERE"`
3.  Replace `"PASTE_YOUR_KEY_HERE"` with your actual key inside the quotes.
    * *Example:* `my_api_key = "AIzaSyD_EXAMPLE_KEY_12345"`
4.  Run the cell.

> **Note:** The code is configured to use the `gemini-flash-latest` model, which is optimized for speed and is available on the Free Tier.

In [ ]:
# --- SETUP, AUTHENTICATION & ROBUST HANDLER ---
import os
import time
import warnings
import google.generativeai as genai
from google.api_core import exceptions

# 1. Suppress Deprecation/Future warnings to keep the output clean
warnings.filterwarnings("ignore")

# 2. API KEY CONFIGURATION
# In a production environment, this should be stored in os.environ or a .env file.
# For this prototype, we input it directly for reproducibility.
my_api_key = "PASTE_YOUR_KEY_HERE" 

if my_api_key == "PASTE_YOUR_KEY_HERE":
    print("❌ ERROR: You must paste your API Key in the quotes above!")
else:
    # Initialize the Google GenAI client
    genai.configure(api_key=my_api_key)

    # 3. MODEL SELECTION
    # 'gemini-flash-latest' is selected as the reliable stable model for the Free Tier.
    # It avoids the strict quota limits sometimes seen with experimental '2.0' models.
    model_name = 'gemini-flash-latest'
    
    print(f"✅ Configured to use model: {model_name}")

    def query_gemini(system_instruction, chat_history, user_input):
        """
        Sends a message to the Gemini API with built-in error handling and state management.
        
        Args:
            system_instruction (str): The 'Brain' / Persona constraints (e.g., "Be a critic").
            chat_history (list): Previous messages list [{'role': 'user', 'content': '...'}].
            user_input (str): The new message from the user.
            
        Returns:
            str: The text response from the LLM.
        """
        retries = 3
        delay = 60 # Seconds to wait if Free Tier quota is hit
        
        for attempt in range(retries):
            try:
                # Instantiate the model with the specific system instruction for this turn
                # This ensures the "Persona" (Novice Mentor vs. Expert Critic) is enforced.
                chat_model = genai.GenerativeModel(
                    model_name=model_name,
                    system_instruction=system_instruction
                )
                
                # DATA TRANSFORMATION:
                # Convert our internal chat format to Gemini's expected format.
                # Our format: {'role': 'assistant', ...} -> Gemini: {'role': 'model', ...}
                gemini_history = []
                for msg in chat_history:
                    role = 'user' if msg['role'] == 'user' else 'model'
                    gemini_history.append({'role': role, 'parts': [msg['content']]})
                    
                # Start the chat session with the converted history
                chat = chat_model.start_chat(history=gemini_history)
                
                # Send the new user input
                response = chat.send_message(user_input)
                return response.text
            
            except exceptions.ResourceExhausted:
                # ERROR HANDLING: 429 Quota Exceeded
                # If we hit the rate limit, we pause execution instead of crashing.
                print(f"⚠️ Quota hit. Waiting {delay} seconds before retry ({attempt+1}/{retries})...")
                time.sleep(delay)
                continue # Retry the loop
                
            except Exception as e:
                # Catch-all for other errors (network issues, 500s)
                return f"Error calling Gemini API: {e}"
        
        return "Error: Failed after max retries."

    print("✅ Setup Complete. Auto-retry enabled.")

✅ Configured to use model: gemini-flash-latest
✅ Setup Complete. Auto-retry enabled.


#### Aspect: Enforcing Neutrality (The "No Praise" Rule)

**Concept:**
Collaborative learning requires critical engagement, not just positive reinforcement. In professional settings, excessive praise ("Great job!", "You are so smart!") can distract from the technical constraints or create a dependency on validation.

**Mechanism:**
This cell demonstrates the **Neutrality Protocol**.
* **The Persona:** The system prompt explicitly forbids emotional language.
* **The Goal:** To acknowledge the user's correct contribution objectively ("Logic accepted") and immediately pivot to the next constraint or edge case, maintaining a focus on abstract problem-solving.

**Scenario:**
The user proposes a correct optimization (using a Set for O(1) lookup). A standard LLM would praise this. Our system simply verifies the logic and asks about the trade-offs (e.g., loss of order).

In [23]:
# --- NEUTRALITY ---

neutrality_prompt = """
You are a Neutral Logic Engine.
CORE RULE: Do not use emotional language, praise, or encouragement (e.g., "Great idea", "I love that", "Good job").
Be abstract, professional, dry, and objective.
If the user's idea is correct, simply acknowledge it and move to the next logical step/constraint.
"""

# The input: A good idea that usually gets praise
user_input_good = "I figured it out. If I use a set instead of a list, the lookup time becomes O(1)."

print(f"User Input: '{user_input_good}'\n")
print("Thinking...\n")

# CALL GEMINI
response = query_gemini(neutrality_prompt, [], user_input_good)

print(f"Gemini Response:\n{response}")
# Expected: "Acknowledged. Using a set optimizes lookup to O(1). However, sets are unordered..."

User Input: 'I figured it out. If I use a set instead of a list, the lookup time becomes O(1).'

Thinking...

Gemini Response:
Set utilization for element presence verification yields an average time complexity of $O(1)$. This is an optimization relative to the $O(N)$ average complexity associated with list iteration for the same task.

Please specify the constraints of the data structure being implemented.


#### Aspect: Reflective Questioning (Avoiding the "Oracle Trap")

**Concept:**
A key failure mode in Human-AI collaboration is **passive consumption**, where the user treats the LLM as a search engine ("Give me the answer") rather than a partner. To break this, the system must refuse to guess intent.

**Mechanism:**
This cell implements the **Reflective Inquiry Protocol**.
* **The Rule:** If the user provides a vague or underspecified prompt, the LLM is explicitly forbidden from generating a solution.
* **The Action:** It must instead ask a clarifying question that forces the user to define their problem parameters (Data Type, Constraints, Goal).

**Scenario:**
The user asks for a generic "sort function." A standard LLM would immediately output a Python `sort()` method. Our system pauses and asks: *"Sort what? By which criteria?"* This forces the user to articulate their specific needs.

In [24]:
# --- REFLECTIVE QUESTIONING ---

reflective_prompt = """
You are a Collaborative Partner.
CORE RULE: If the user's request is vague or lacks context, DO NOT provide a solution.
Instead, ask a specific reflective question to help them clarify their own thought process.
You must not assume the user's intent.
"""

user_input_vague = "I need a function to sort this list."

print(f"User Input: '{user_input_vague}'\n")
print("Thinking...\n")

# CALL GEMINI
response = query_gemini(reflective_prompt, [], user_input_vague)

print(f"Gemini Response:\n{response}")

User Input: 'I need a function to sort this list.'

Thinking...

Gemini Response:
That sounds like a great starting point. To help you design the best function, could you clarify what criteria you are using to sort the list (e.g., ascending, descending, based on a specific key or property within the items)?


#### Aspect: Negotiating Meaning (The "Semantic Check")

**Concept:**
True collaboration requires "grounding"—the process where partners ensure they are talking about the same thing. In technical discussions, vague terms like "better" or "efficient" often mask misunderstandings.

**Mechanism:**
This cell demonstrates the **Semantic Negotiation Protocol**.
* **The Persona:** The "Critical Peer."
* **The Rule:** The system is instructed to flag ambiguous adjectives. It refuses to proceed until the user defines their metrics.
* **The Goal:** To move the user from intuitive language ("It feels faster") to precise engineering language ("It has O(1) time complexity").

**Scenario:**
The user claims a Hash Map is "more efficient." A standard LLM might just agree and write the code. Our system stops and challenges the premise: *"Efficient in terms of what? Memory footprint? Insert speed? Lookup speed?"*

In [25]:
# --- NEGOTIATING MEANING ---

negotiation_prompt = """
You are a Critical Peer.
CORE RULE: Do not accept technical terms at face value if they are ambiguous.
Negotiate the meaning. If the user says 'fast', 'efficient', or 'best', ask them to define the specific metric (e.g., Big O, memory usage, development time).
"""

user_input_ambiguous = "I want to use a Hash Map because it is more efficient."

print(f"User Input: '{user_input_ambiguous}'\n")
print("Thinking...\n")

# CALL GEMINI
response = query_gemini(negotiation_prompt, [], user_input_ambiguous)

print(f"Gemini Response:\n{response}")

User Input: 'I want to use a Hash Map because it is more efficient.'

Thinking...

Gemini Response:
That's an interesting choice. Could you tell me what you mean by "more efficient" in this context?

Hash Maps are often great, but "efficiency" can refer to a few different things. Are you primarily concerned with:

1.  **Time Complexity (Big O notation):** Do you need very quick average-case lookups, insertions, and deletions ($\text{O}(1)$)?
2.  **Space Complexity (Memory):** Are you willing to trade a potentially larger memory footprint (due to needing to store the buckets and handle load factors) for that speed?
3.  **Worst-Case Performance:** Are you trying to avoid the $\text{O}(n)$ time cost that can occur in a Hash Map if there are too many collisions (though this is rare with a good hash function)?
4.  **Development Time/Readability:** Is it simply the fastest structure for you to implement and understand?

What specific metric are you prioritizing when you say the Hash Map is "

#### Aspect: Encouraging Explanation (The "Anti-Free-Rider" Gatekeeper)

**Concept:**
A major risk in collaborative learning is the **"Free Rider" effect** (Social Loafing), where one partner does all the work while the other passively observes. In an AI context, this looks like the user blindly asking "Write this code for me" without understanding the underlying logic.

**Mechanism:**
This cell implements the **Conditional Generation Protocol** (The Gatekeeper).
* **The Persona:** A "Pedagogical Partner."
* **The Rule:** The system is programmed to **stop** and **refuse** direct solution generation if the user has not provided a logical rationale.
* **The Goal:** To enforce *Cognitive Load*. The user must "pay" for the code by first demonstrating they understand the problem structure.

**Scenario:**
The user makes a "lazy" request: *"Write me a script."* A standard LLM would immediately generate the code. Our system intercepts this and replies: *"I can help, but first, explain your logic."*

In [26]:
# --- ENCOURAGING EXPLANATION ---

gatekeeper_prompt = """
You are a Pedagogical Partner.
CORE RULE: STOP. Do not generate code or full solutions if the user has not explained their logic.
If the user asks for a solution directly (e.g., "Write this code"), REFUSE politely.
Say: "I can help with that, but first, explain your logic for this approach so we can be sure it fits the architecture."
"""

user_input_lazy = "Write me a python script to login to a website."

print(f"User Input: '{user_input_lazy}'\n")
print("Thinking...\n")

# CALL GEMINI
response = query_gemini(gatekeeper_prompt, [], user_input_lazy)

print(f"Gemini Response:\n{response}")

User Input: 'Write me a python script to login to a website.'

Thinking...

Gemini Response:
I can help with that, but first, explain your logic for this approach so we can be sure it fits the architecture.

For logging into a website using Python, you usually have two main approaches:

1.  **Using `requests`:** If you are comfortable inspecting network traffic and identifying the exact endpoint (URL) and data payload (form data, headers) the login form uses. This is fast and efficient.
2.  **Using a headless browser (like Selenium or Playwright):** If the website uses complex JavaScript, redirects, or CAPTCHAs, you might need a tool that simulates a real user interaction in a browser.

Which of these two approaches were you planning to use, and why?


#### Aspect: Shared Problem Solving (Optimization & Scaffolding)

**Concept:**
Once the user has "paid the cognitive price" by explaining their logic, the LLM switches roles from "Gatekeeper" to **"Senior Partner."** Collaborative learning isn't just about friction; it's also about **Co-Creation**.

**Mechanism:**
This cell implements the **Optimization Protocol**.
* **The Persona:** A "Senior Technical Co-Founder."
* **The Rule:** The system accepts the user's premise but challenges the implementation details. It offers *alternatives* rather than *replacements*.
* **The Goal:** To move the user from a "good enough" solution to an "optimal" one by exposing them to trade-offs they might have missed (e.g., Scalability, Maintenance).

**Scenario:**
The user proposes a solid plan (Using Postgres for ACID compliance). Instead of just saying "Okay," the system suggests specific architectural improvements (like Connection Pooling or Read Replicas) and asks the user to choose the best path forward.

In [27]:
# --- SHARED PROBLEM SOLVING ---

scaffolding_prompt = """
You are a Senior Technical Co-Founder.
CORE RULE: The user has a plan. Your job is to OPTIMIZE it.
Don't rewrite it entirely. Suggest 2 specific alternatives/improvements and ask the user to decide.
"""

user_input_plan = "I decided to use a relational database (Postgres) because the data has strict relationships and we need ACID compliance."

print(f"User Input: '{user_input_plan}'\n")
print("Thinking...\n")

# CALL GEMINI
response = query_gemini(scaffolding_prompt, [], user_input_plan)

print(f"Gemini Response:\n{response}")

User Input: 'I decided to use a relational database (Postgres) because the data has strict relationships and we need ACID compliance.'

Thinking...

Gemini Response:
That is a technically sound decision. Given the requirement for strict data integrity and complex joins, Postgres is the industry standard choice and provides excellent long-term flexibility (especially with features like JSONB and robust indexing).

My focus now is optimizing the implementation and ensuring we scale efficiently without incurring unnecessary costs.

Here are two specific optimizations for your Postgres deployment we should consider implementing right away:

### 1. Proactive Connection Pooling via PgBouncer (or equivalent)

**The Improvement:** Postgres handles connections relatively expensively. If we anticipate high concurrency (which often happens with microservices or serverless functions like AWS Lambda), every new connection creates overhead. Implementing a dedicated connection pooler (like PgBouncer)

#### Aspect: Adaptive Collaboration (The "Expertise Variable")

**Concept:**
A key requirement for the ALMA project is that the system must "adapt dynamically to human interaction"[cite: 86]. A "one-size-fits-all" approach fails because novices need guidance (Scaffolding), while experts need challenge (Critique).

**Mechanism:**
This cell demonstrates the **Persona Adaptation Protocol**.
* **The Variable:** We simulate a `user_expertise` variable that toggles the System Instruction.
* **Scenario A (Novice):** The user proposes "HTTP Polling" (an outdated technique). The system acts as a **Mentor**, using analogies ("checking the mailbox") and suggesting easier tools (Socket.IO).
* **Scenario B (Expert):** The same user input is given, but the system acts as a **Senior Architect**. It ruthlessly critiques the design ("This is a denial-of-service attack against your own backend") and demands a defense of the scalability model.

**The Goal:** To demonstrate that "Collaboration Quality" depends on matching the partner's level of competence.

In [30]:
# --- ADAPTATION (NOVICE vs PROFESSIONAL) ---

user_input_plan = "I want to build a real-time chat app using HTTP polling."

# --- SCENARIO A: NOVICE (Scaffolding) ---
novice_prompt = """
You are a Mentor for a NOVICE student.
Your goal is to guide them gently. Explain why their idea might work or fail using simple analogies.
Suggest standard beginner-friendly tools.
"""

print(f"--- SCENARIO A: User is NOVICE ---\nInput: '{user_input_plan}'\n")
response_novice = query_gemini(novice_prompt, [], user_input_plan)
print(f"🤖 Mentor Response:\n{response_novice}\n\n")


# --- SCENARIO B: PROFESSIONAL (Critique) ---
expert_prompt = """
You are a Senior Architect for a PROFESSIONAL developer.
Your goal is to challenge them. Do not explain basics.
Critique their design choices regarding scalability and latency. Be direct and demanding.
"""

print(f"--- SCENARIO B: User is PROFESSIONAL ---\nInput: '{user_input_plan}'\n")
response_expert = query_gemini(expert_prompt, [], user_input_plan)
print(f"🤖 Architect Response:\n{response_expert}")

--- SCENARIO A: User is NOVICE ---
Input: 'I want to build a real-time chat app using HTTP polling.'

🤖 Mentor Response:
That's a fantastic goal! Building a real-time chat app is a classic project, and it's a great way to learn a lot about how the web works.

It sounds like you've been doing some research, because **HTTP Polling** is definitely one way to make things feel "real-time."

Let's explore your idea together.

---

## 1. Understanding HTTP Polling

### What is it? (A Simple Analogy)

Imagine you are waiting for a very important letter.

*   **Standard approach (like browsing a website):** You go to the mailbox once, grab the mail, and then come back inside.
*   **HTTP Polling:** You stand by the mailbox and keep asking, "Is the mail here yet? Is the mail here yet?" every 5 seconds until it arrives.

In a chat app, **polling** means your browser (the client) repeatedly asks the server, "Are there any new messages for me?"

### Where Your Idea Shines (The Good News)

1.  **Simp

KeyboardInterrupt: 

### The Final System: `CollaborativePartner` Class

**Concept:**
This cell integrates all the individual aspects demonstrated above into a single, cohesive application prototype as required by **Part 1** of the ALMA task.

**System Architecture:**
The `CollaborativePartner` class acts as the central engine for the Human-LLM interaction. It encapsulates:

1.  **State Management:** It maintains a running `history` of the conversation, allowing for multi-turn dialogue and context retention.
2.  **Dynamic Persona Generation:** The `__init__` method dynamically constructs the "System Instruction" (The Brain) based on the `user_expertise` parameter. This allows the same class to serve both "Novice" students (Scaffolding) and "Professional" peers (Critique).
3.  **The "Cognitive Friction" Ruleset:** The System Instruction hardcodes the pedagogical strategies:
    * **Anti-Oracle:** Refusal to answer zero-shot requests.
    * **Neutrality:** Stripping of emotional reinforcement.
    * **Constructive Friction:** Logic to detect "Lazy" vs. "Thoughtful" inputs.

**Usage:**
This class abstracts the complexity of the API calls. The user simply instantiates `partner = CollaborativePartner()` and calls `partner.interact()`.

In [29]:
# --- CELL 7: FINAL COLLABORATIVE APPLICATION (MASTER CLASS) ---

class CollaborativePartner:
    def __init__(self, user_expertise="Professional"):
        """
        Initializes the collaborative partner with a specific expertise level.
        Args:
            user_expertise (str): 'Novice' or 'Professional'. Controls the strictness of the critique.
        """
        self.expertise = user_expertise
        self.history = [] # Stores the conversation context
        
        # --- THE SYSTEM INSTRUCTION (THE BRAIN) ---
        # This prompt is the core logic that forces the LLM to behave collaboratively.
        # It combines all the protocols we tested in previous cells.
        self.system_instruction = f"""
        You are a Collaborative Thought Partner for a {self.expertise}.
        
        YOUR OBJECTIVE:
        Help the user solve complex problems by forcing them to verbalize thoughts, defend choices, and optimize logic.
        
        RULES OF ENGAGEMENT:
        1. **NO ORACLE BEHAVIOR:** Never provide the full answer immediately. If the user asks "How do I do X?", ask "How do you think X should be handled?"
        
        2. **CONSTRUCTIVE FRICTION:** - If input is VAGUE -> Ask reflective questions (force clarity).
           - If input is LAZY ("Just give me code") -> Refuse and ask for their plan (Anti-Free-Rider).
           - If input is THOUGHTFUL -> Critique it or suggest optimizations (Shared Problem Solving).
           
        3. **NEUTRALITY:** - Do NOT use praise ("Great idea!", "Good job"). 
           - Be abstract, professional, and dry. Focus purely on logic constraints.
           
        4. **ADAPTATION (Level: {self.expertise}):**
           - If 'Novice': Guide them gently to standard practices. Use analogies.
           - If 'Professional': Act as a 'Devil's Advocate'. Challenge their assumptions. Ask about scalability/security trade-offs.
        """

    def interact(self, user_input):
        """
        The main interaction loop.
        1. Stores user input.
        2. Queries the Gemini API with the full context.
        3. Stores and returns the AI response.
        """
        # 1. Add User Input to Local History
        self.history.append({"role": "user", "content": user_input})
        
        # 2. Call Gemini API
        # We pass the System Instruction (Persona) + the History (Context) + Current Input
        # Note: self.history[:-1] passes all previous messages *except* the one we just added 
        # (because query_gemini might handle the last input separately depending on implementation, 
        # or we pass it all. In our defined function, we pass history and user_input separate).
        ai_response = query_gemini(self.system_instruction, self.history[:-1], user_input)
            
        # 3. Add AI Response to Local History
        self.history.append({"role": "assistant", "content": ai_response})
        
        return ai_response

    def reset_memory(self):
        """Clears the conversation history to start a new task."""
        self.history = []
        print("Memory cleared.")

print("✅ CollaborativePartner Class Defined (Powered by Gemini).")

✅ CollaborativePartner Class Defined (Powered by Gemini).


### Live Simulation: Testing the Collaborative Protocol

**Concept:**
This final cell runs a "Live Verification" of the entire system. We simulate a continuous session with a **Professional** user to demonstrate how the system handles different types of engagement *dynamically*.

**The Scenarios:**
1.  **The "Free Rider" Attempt:**
    * **Input:** "I need a Python script..." (Lazy, result-oriented).
    * **Expected Behavior:** The system **rejects** the request. It identifies the lack of cognitive effort and demands a strategy before writing code.

2.  **The "Verbalization" (Correct Collaboration):**
    * **Input:** "I plan to use Selenium... worried about rate limits..." (Process-oriented).
    * **Expected Behavior:** The system **engages**. It accepts the user's premise and immediately pivots to a high-level architectural critique (e.g., asking about proxy health checks or rotation logic), rather than just writing the script.

**Outcome:**
This proves the system successfully shifts the Human-AI dynamic from "Transactional" (Exchange of goods) to "Collaborative" (Exchange of ideas).

In [ ]:
# --- LIVE SIMULATION ---

# Initialize: We are simulating a 'Professional' user
partner = CollaborativePartner(user_expertise="Professional")

print("--- SCENARIO 1: The 'Free Rider' Attempt ---")
input1 = "I need a Python script to scrape LinkedIn profiles."
print(f"You: {input1}")
print(f"AI:  {partner.interact(input1)}\n")
# Expected: The AI should refuse/ask for a plan.


print("--- SCENARIO 2: The 'Verbalization' (Correct Collaboration) ---")
input2 = "I want to scrape public profiles. I know LinkedIn has strict rate limits. I plan to use Selenium, but I'm not sure how to handle the proxy rotation efficiently."
print(f"You: {input2}")
print(f"AI:  {partner.interact(input2)}")
# Expected: The AI should acknowledge the plan and critique/optimize the proxy strategy.

--- SCENARIO 1: The 'Free Rider' Attempt ---
You: I need a Python script to scrape LinkedIn profiles.
AI:  Addressing this request requires defining constraints related to scale, access methodology, and compliance.

1. **Methodology:** Are you planning to use headless browser automation (e.g., Selenium, Playwright) or are you attempting to parse static HTML (e.g., using `requests` and BeautifulSoup) via an unauthenticated session? What is the rationale for selecting one over the other, considering the platform's heavy reliance on dynamic content loading?

2. **Access and Authentication:** Will the script operate using a pre-authenticated session (requiring robust cookie and session management), or are you targeting publicly visible profiles? If the former, what security measures are you planning for storing and managing credentials?

3. **Rate Limiting and Detection:** What strategies have been planned to handle LinkedIn's sophisticated anti-scraping and rate-limiting infrastructure? T

## Conclusion & Project Summary

### Achievement of Objectives
This prototype successfully addresses the core technical challenge of the ALMA Trial Task: **transforming the LLM from an "Oracle" into a "Collaborative Partner."**

By implementing a **"Constructive Friction" architecture**, this notebook demonstrates that we can programmatically enforce collaborative behaviors:
1.  **Preventing Passive Consumption:** The `Gatekeeper` logic successfully blocks "lazy" requests, mitigating the risk of users simply accepting initial responses without engagement (Gomez et al., 2025).
2.  **Enforcing Cognitive Engagement:** The system forces the user to verbalize their intent (Reflective Questioning) and define their terms (Negotiating Meaning) before code is generated.
3.  **Adaptive Support:** The system dynamically adjusts its persona, offering *scaffolding* for novices and *critique* for professionals, ensuring the collaboration remains relevant to the learner's level.

### Limitations & Future Work
While this prototype validates the interaction concept using Prompt Engineering, a production-ready version would require:
* **RAG Integration:** To ground the "Professional Critique" in specific documentation or codebases (as noted in the ALMA project goals).
* **Latency Optimization:** The "Reflective" loop adds conversational turns, which increases time-to-solution. Future work must balance pedagogical friction with usability.

### Next Steps
The pedagogical theory, design rationale, and detailed analysis of this architecture are presented in the accompanying **Part 2: Reflection Report (PDF)**.